### Connecting to BigQuery

Before running queries, we first need to set up a connection to **Google BigQuery** using the Python client library.  
This involves:

1. Importing the `google.cloud.bigquery` module.  
2. Creating a BigQuery client object, which handles authentication and connections.  
3. Defining a reference to the **Chicago Crime** public dataset hosted by Google.  
4. Fetching the dataset object, which allows us to explore its tables and metadata.  

In [1]:
# import the BigQuery client library
from google.cloud import bigquery

# create a BigQuery client object
client = bigquery.Client()

# define a reference to the public dataset
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

# fetch the dataset
dataset = client.get_dataset(dataset_ref)

/home/jd2208/anaconda3/envs/datanalysis/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jd2208/anaconda3/envs/datanalysis/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Listing Tables in the Dataset

Once we have connected to the dataset, the next step is to explore its contents.  
Using the BigQuery client, we can list all the tables available in the dataset. This helps us understand the structure of the dataset before running queries. This shows us that there is only one table in the database.

In [3]:
# list all tables in the dataset
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

crime


### Fetching the Table Schema

To understand the structure of the `crime` table, we retrieve its schema.  
The schema provides details about each column, such as name, type, and mode (nullable or required).  

This is useful to know what fields are available before writing queries.

In [4]:
# fetch the schema of a specific table
table_ref = dataset.table("crime")
table = client.get_table(table_ref)
table.schema

[SchemaField('unique_key', 'INTEGER', 'REQUIRED', None, None, (), None),
 SchemaField('case_number', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('block', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('iucr', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('primary_type', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('description', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('location_description', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('arrest', 'BOOLEAN', 'NULLABLE', None, None, (), None),
 SchemaField('domestic', 'BOOLEAN', 'NULLABLE', None, None, (), None),
 SchemaField('beat', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('district', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('ward', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('community_area', 'INTEGER', 'NULLABLE', None, None, (), None),
 

### Previewing Table Data  

We fetch the first five rows from the `crime` table and load them into a pandas DataFrame. This gives a quick look at the data values and helps verify the table structure.


In [5]:
# print the first few rows of the table
client.list_rows(table, max_results=5).to_dataframe()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,11060331,JA397659,2017-08-13 12:27:00+00:00,0000X W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,42,32,02,1176068.0,1901284.0,2017,2018-02-10 15:50:01+00:00,41.884486,-87.628909,"(41.884486217, -87.628908795)"
1,7263938,HR678180,2009-12-06 03:00:00+00:00,006XX W WASHINGTON BLVD,0313,ROBBERY,ARMED: OTHER DANGEROUS WEAPON,PARKING LOT/GARAGE(NON.RESID.),False,False,...,27,28,03,1171769.0,1900767.0,2009,2018-02-28 15:56:25+00:00,41.883163,-87.644710,"(41.883163285, -87.644710386)"
2,2431199,HH749112,2002-10-30 21:59:19+00:00,005XX W MADISON ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,...,42,28,03,1173039.0,1900311.0,2002,2018-02-28 15:56:25+00:00,41.881884,-87.640060,"(41.881883933, -87.640060441)"
3,13141030,JG340094,2023-07-13 22:00:00+00:00,002XX N STATE ST,0320,ROBBERY,STRONG ARM - NO WEAPON,CTA PLATFORM,False,False,...,42,32,03,1176327.0,1901797.0,2023,2023-08-19 15:40:26+00:00,41.885888,-87.627942,"(41.885888079, -87.627942238)"
4,2314598,HH602912,2002-08-24 22:45:00+00:00,003XX W WASHINGTON ST,0320,ROBBERY,STRONGARM - NO WEAPON,STREET,False,False,...,42,32,03,1174205.0,1900802.0,2002,2018-02-28 15:56:25+00:00,41.883205,-87.635764,"(41.88320534, -87.635764295)"


### Filtering Crimes by Type  

We run a SQL query to select only records where the `primary_type` is **"CRIM SEXUAL ASSAULT"** from the `crime` table.  
The results are returned as a pandas DataFrame, allowing further inspection and analysis in Python.

In [ ]:
# Select crime was "CRIM SEXUAL ASSAULT" by query
query = """
        SELECT primary_type
        FROM `bigquery-public-data.chicago_crime.crime`
        WHERE primary_type = 'CRIM SEXUAL ASSAULT'
        """
# Set up the query job
query_job = client.query(query)
# Execute the query and convert the results to a pandas DataFrame
crime_sexual_assault = query_job.to_dataframe()
crime_sexual_assault.head()

,primary_type
0,CRIM SEXUAL ASSAULT
1,CRIM SEXUAL ASSAULT
2,CRIM SEXUAL ASSAULT
3,CRIM SEXUAL ASSAULT
4,CRIM SEXUAL ASSAULT


### Estimating Query Size  

Before executing the query, we perform a **dry run** to estimate how much data it will process.  
This step is important because in BigQuery, **query cost depends on the amount of data scanned**.  
By checking the estimated size (in gigabytes), we can avoid unexpectedly large or expensive queries.


In [ ]:
# estimate the size of query
query = """
        SELECT primary_type, location_description
        FROM `bigquery-public-data.chicago_crime.crime`
        WHERE primary_type = 'CRIM SEXUAL ASSAULT'
        """
# Create a job configuration with dry_run set to True
job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
# Execute the query with the job configuration
query_job = client.query(query, job_config=job_config)
# Print the estimated number of Gigabytes the query will process
query_bytes = query_job.total_bytes_processed
query_gigabytes = query_bytes / 1.0e9
print(f"This query will process approximately {query_gigabytes:.2f} GB of data")

This query will process approximately 0.21 GB of data


### Exploratory Question:  
**How many different types of crimes are recorded in the dataset, and which are the most common?**

Rta: There are 34 types of crimes reported, with theft being the most common. 


In [ ]:
query_crimes = """
        SELECT primary_type, COUNT(1) as number_of_crimes
        FROM `bigquery-public-data.chicago_crime.crime`
        GROUP BY primary_type
        ORDER BY number_of_crimes DESC
        """
# Execute the query and convert the results to a pandas DataFrame
query_job = client.query(query_crimes)
crimes = query_job.to_dataframe()
crimes.head(-1)

,primary_type,number_of_crimes
0,THEFT,1773570
1,BATTERY,1523452
2,CRIMINAL DAMAGE,950728
3,NARCOTICS,761985
4,ASSAULT,558450
5,OTHER OFFENSE,521033
6,BURGLARY,442506
7,MOTOR VEHICLE THEFT,426067
8,DECEPTIVE PRACTICE,383734
9,ROBBERY,313036


### Exploratory Question:  
**How many crimes occurred in Community Area 23, and which are the most common?**

Rta: The most common crime in area 23 is narcotics

In [ ]:
query_area23 = """
        SELECT primary_type, community_area, COUNT(1) as number_of_crimes
        FROM `bigquery-public-data.chicago_crime.crime`
        GROUP BY primary_type, community_area
        HAVING community_area = 23
        ORDER BY number_of_crimes DESC"""
# Execute the query and convert the results to a pandas DataFrame
query_job = client.query(query_area23)
area23 = query_job.to_dataframe()
area23.head(-1)

,primary_type,community_area,number_of_crimes
0,NARCOTICS,23,52624
1,BATTERY,23,46990
2,THEFT,23,31756
3,CRIMINAL DAMAGE,23,22261
4,ASSAULT,23,15969
5,OTHER OFFENSE,23,13134
6,MOTOR VEHICLE THEFT,23,11695
7,ROBBERY,23,9551
8,BURGLARY,23,9420
9,DECEPTIVE PRACTICE,23,5884


### Exploratory Question:  
**On which day of the week do thefts occur most frequently?**

RTA: The day when most crimes occur is the 6th day, that is, Saturdays.

In [ ]:
query_timethefts = """
                SELECT PRIMARY_TYPE, COUNT(1) AS NUMBER_OF_CRIMES,
                EXTRACT(DAYOFWEEK FROM date) AS day_of_week
                FROM `bigquery-public-data.chicago_crime.crime`
                WHERE PRIMARY_TYPE = 'THEFT'
                GROUP BY day_of_week, PRIMARY_TYPE
                ORDER BY NUMBER_OF_CRIMES DESC
                """
# Execute the query and convert the results to a pandas DataFrame
query_job = client.query(query_timethefts)
thefts = query_job.to_dataframe()
thefts.head(-1)

,PRIMARY_TYPE,NUMBER_OF_CRIMES,day_of_week
0,THEFT,273627,6
1,THEFT,258238,4
2,THEFT,256694,3
3,THEFT,255391,5
4,THEFT,255301,2
5,THEFT,252167,7
